In [1]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import statistics

In [2]:
ltable = pd.read_csv('ltable.csv', encoding = "ISO-8859-1")
rtable = pd.read_csv('rtable.csv', encoding = "ISO-8859-1")
sample = pd.read_csv('sample_submission.csv')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
ltable.head()

,ltable_id,title,category,brand,modelno,price
0,0,draper infrared remote transmitter,electronics - general,draper,121066,58.45
1,1,epson 1500 hours 200w uhe projector lamp elplp12,monitors,epson,elplp12,438.84
2,2,comprehensive two-piece 75 precision bnc jack ...,tv accessories,comprehensive,bj-2c7559,59.25
3,3,d-link dcs-1100 network camera,garden - general,d-link,dcs-1100,99.82
4,4,startech.com rkpw247015 24 outlet power strip,electronics - general,startech,rkpw247015,59.00


In [4]:
rtable.head()

,rtable_id,title,category,brand,modelno,price
0,0,koss eq50 3-band stereo equalizer,headphone accessories,koss,152132,12.65
1,1,kodak black ink cartridge 10b 1163641,inkjet printer ink,kodak,1163641,10.28
2,2,kingston 128mx64 pc2700 compaq evo d320 ktc-d3...,computers accessories,kingston,ktc-d320 / 1g,33.75
3,3,kinamax ms-ues2 mini high precision usb 3-butt...,mice,kinamax,ms-ues2,6.99
4,4,kensington k72349us wireless mouse for netbooks,mice,kensington,k72349us,24.00


In [5]:
sample.head()

,id,label
0,0,0
1,8,0
2,9,1
3,10,1
4,15,1


In [6]:
train.head()

,ltable_id,rtable_id,label,id
0,621,3167,0,1
1,2115,8789,0,2
2,278,9064,0,3
3,1673,13279,0,4
4,2305,2948,0,5


In [7]:
test.head()

,ltable_id,rtable_id,id
0,1853,2139,0
1,1718,11835,8
2,1624,5013,9
3,614,5969,10
4,1512,19684,15


In [8]:
ltable.query("ltable_id == 1512")[['ltable_id', 'title', 'category', 'brand', 'modelno', 'price']]

,ltable_id,title,category,brand,modelno,price
1512,1512,uniden d1680 dect 6.0 with caller id and call ...,electronics - general,uniden,d1680,33.84


In [9]:
rtable.query("rtable_id == 19684")[['rtable_id', 'title', 'category', 'brand', 'modelno', 'price']]

,rtable_id,title,category,brand,modelno,price
19684,19684,new-uniden d1660 dect 6.0 cordless phone syste...,cordless telephones,uniden,NaN,24.09


#### Our goal is to check whether the entities match in the test set, as we have the train set, we can analyze the train set first to determine the threshold

In [10]:
train_not_match = train.query("label == 0")[['ltable_id', 'rtable_id']]

In [11]:
train_match = train.query("label == 1")[['ltable_id', 'rtable_id']]

In [12]:
train_match.head()

,ltable_id,rtable_id
6,88,3269
9,470,10874
15,1534,1090
63,99,4520
83,498,5577


In [13]:
train['ltable_id'][0]

621

In [14]:
ltable.loc[ltable['ltable_id'] == train['ltable_id'][0]]['title']

621    zotac geforce gt430 1gb ddr3 pci-express 2.0 g...
Name: title, dtype: object

In [15]:
fuzz.token_sort_ratio(ltable.loc[ltable['ltable_id'] == train['ltable_id'][0]]['title'], rtable.loc[rtable['rtable_id'] == train['rtable_id'][0]]['title'])

63

In [16]:
# Find the train entities match with the aggregated lowest score
train_match_score = []
for i in range(len(train_match)):
      train_match_score.append((fuzz.token_sort_ratio(ltable.loc[ltable['ltable_id'] == train['ltable_id'][i]]['title'], rtable.loc[rtable['rtable_id'] == train['rtable_id'][i]]['title'])
                              + fuzz.token_sort_ratio(ltable.loc[ltable['ltable_id'] == train['ltable_id'][i]]['brand'], rtable.loc[rtable['rtable_id'] == train['rtable_id'][i]]['brand'])
                              + fuzz.token_sort_ratio(ltable.loc[ltable['ltable_id'] == train['ltable_id'][i]]['category'], rtable.loc[rtable['rtable_id'] == train['rtable_id'][i]]['category'])
                              + fuzz.token_sort_ratio(ltable.loc[ltable['ltable_id'] == train['ltable_id'][i]]['modelno'], rtable.loc[rtable['rtable_id'] == train['rtable_id'][i]]['modelno']))/4)

In [17]:
# Find the train entities match with the aggregated lowest score
train_match_score = []
for i in range(len(train_not_match)):
      train_match_score.append(fuzz.token_sort_ratio(ltable.loc[ltable['ltable_id'] == train['ltable_id'][i]]['title'], rtable.loc[rtable['rtable_id'] == train['rtable_id'][i]]['title']))

In [18]:
# Find the threshold
statistics.mean(train_match_score)
#min(train_match_score)

76.91038318912237

#### start to predict labels

In [19]:
label = []
for i in range(len(test)):
    score = fuzz.token_set_ratio(ltable.loc[ltable['ltable_id'] == test['ltable_id'][i]]['title'], rtable.loc[rtable['rtable_id'] == test['rtable_id'][i]]['title'])
    #print(score)
    if score >= 88:
        label.append(1)
    else:
        label.append(0)
        
     

In [20]:
len(label)

3093

In [21]:
test['id'].values

array([    0,     8,     9, ..., 10223, 10225, 10237])

In [22]:
predict = {}
predict['id'] = test['id'].values
predict['label'] = np.asarray(label)

In [23]:
predict = pd.DataFrame(predict)

In [24]:
predict.to_csv('submission.csv', index=False)

In [25]:
label

[0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
